# imports

In [23]:
import datetime
import importlib
import pandas as pd, numpy as np
import scipy
import matplotlib.pyplot as plt
from numpy_ext import rolling_apply as rolling_apply_ext
import talib
from talib import MA_Type
import numpy.linalg

In [24]:
import algo.data.read
importlib.reload(algo.data.read)

<module 'algo.data.read' from '/home/junlim/projects/mean_reversal/algo/data/read.py'>

In [25]:
symbols_pool = [s for _, s in sorted([(m, s) for s, m in algo.data.read.df_gemini_202310.mean().items() if m > 10 and m < 500], reverse=True)]
# remove duplicates within USDT / BUSD swap
symbols_pool = [s for s in symbols_pool if s not in [s.replace('USDT', 'BUSD') for s in symbols_pool if 'USDT' in s]]
symbols_pool

['BCH/USD',
 'QNT/USD',
 'AAVE/USD',
 'LTC/USD',
 'COMP/USD',
 'SOL/USD',
 'ZEC/USD',
 'INJ/USD',
 'LINK/USD',
 'AVAX/USD']

In [26]:
algo.data.read.df_gemini_202309

,BTC/USD,ETH/USD,ZEC/USD,BCH/USD,LTC/USD,BAT/USD,DAI/USD,LINK/USD,OXT/USD,AMP/USD,...,SAMO/USD,IMX/USD,IOTX/USD,BUSD/USD,AVAX/USD,ATOM/USD,USDT/USD,PEPE/USD,XRP/USD,HNT/USD
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-09-01 00:00:00+00:00,34506.42,1826.82,27.35,260.6,70.8,0.20364,0.99969,10.3111,0.06664,0.0016,...,0.003858,0.63332,0.020177,0.99861,10.306,6.818,1.0002,9.340000e-07,0.56059,1.5534
2023-09-01 00:01:00+00:00,34506.42,1826.82,27.35,260.6,70.8,0.20364,0.99969,10.3111,0.06664,0.0016,...,0.003858,0.63332,0.020177,0.99861,10.306,6.818,1.0002,9.340000e-07,0.56059,1.5534
2023-09-01 00:02:00+00:00,34506.42,1826.82,27.35,260.6,70.8,0.20364,0.99969,10.3111,0.06664,0.0016,...,0.003858,0.63332,0.020177,0.99861,10.306,6.818,1.0002,9.340000e-07,0.56059,1.5534
2023-09-01 00:03:00+00:00,34506.42,1826.82,27.35,260.6,70.8,0.20364,0.99969,10.3111,0.06664,0.0016,...,0.003858,0.63332,0.020177,0.99861,10.306,6.818,1.0002,9.340000e-07,0.56059,1.5534
2023-09-01 00:04:00+00:00,34506.42,1826.82,27.35,260.6,70.8,0.20364,0.99969,10.3111,0.06664,0.0016,...,0.003858,0.63332,0.020177,0.99861,10.306,6.818,1.0002,9.340000e-07,0.56059,1.5534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-30 11:55:00+00:00,34506.42,1826.82,27.35,260.6,70.8,0.20364,0.99969,10.3111,0.06664,0.0016,...,0.003858,0.63332,0.020177,0.99861,10.306,6.818,1.0002,9.340000e-07,0.56059,1.5534
2023-09-30 11:56:00+00:00,34506.42,1826.82,27.35,260.6,70.8,0.20364,0.99969,10.3111,0.06664,0.0016,...,0.003858,0.63332,0.020177,0.99861,10.306,6.818,1.0002,9.340000e-07,0.56059,1.5534
2023-09-30 11:57:00+00:00,34506.42,1826.82,27.35,260.6,70.8,0.20364,0.99969,10.3111,0.06664,0.0016,...,0.003858,0.63332,0.020177,0.99861,10.306,6.818,1.0002,9.340000e-07,0.56059,1.5534


In [35]:
set_size, set_num = 2, 6
symbols_sets = algo.data.read.get_high_corr_symbols_set_of(algo.data.read.df_gemini_202309, 10, set_size, set_num, symbols_pool, high_corr=True)
symbols_sets

start_symbol: QNT/USD
(nan, 'QNT/USD', 'BCH/USD')
start_symbol: LTC/USD
(nan, 'LTC/USD', 'AAVE/USD')
start_symbol: SOL/USD
(nan, 'SOL/USD', 'COMP/USD')
start_symbol: INJ/USD
(nan, 'INJ/USD', 'ZEC/USD')
start_symbol: AVAX/USD
(nan, 'AVAX/USD', 'LINK/USD')


IndexError: list index out of range

In [ ]:
set_size, set_num = 2, 6
symbols_sets_low_corr = algo.data.read.get_high_corr_symbols_set_of(algo.data.read.df_gemini_202309, 10, set_size, set_num, symbols_pool, high_corr=False)
symbols_sets_low_corr

# Solve the minimal predictability condition

In [31]:
import algo.minimal_predictability.calculate
importlib.reload(algo.minimal_predictability.calculate)

<module 'algo.minimal_predictability.calculate' from '/home/junlim/projects/mean_reversal/algo/minimal_predictability/calculate.py'>

# Trading calculations

In [32]:
import algo.statarbitrage.bband
importlib.reload(algo.statarbitrage.bband)

<module 'algo.statarbitrage.bband' from '/home/junlim/projects/mean_reversal/algo/statarbitrage/bband.py'>

# Rolling-based Portfolio Construction

In [33]:
import algo.research.simulation
importlib.reload(algo.research.simulation)

<module 'algo.research.simulation' from '/home/junlim/projects/mean_reversal/algo/research/simulation.py'>

In [34]:
bband_trading_param = algo.statarbitrage.bband.BBandTradingParam(60, 2.0)
stat_arbitrage_trading_param = algo.research.simulation.StatArbitrageTradingParam(10, 180, 120, bband_trading_param)
for symbols in symbols_sets[:]:
    print(f'symbols: {symbols}')
    values_list = algo.research.simulation.get_trading_result(algo.data.read.df_gemini_202310, symbols, stat_arbitrage_trading_param, if_evecs=True)
    values_concat = pd.concat(values_list)
    ax = values_concat.profit_raw.cumsum().plot(figsize=(12, 1))
    ymin, ymax = 0, values_concat.profit_raw.cumsum().max()
    
    #ax.vlines(x=list(df_rolling_wgt_resampled.index), ymin=ymin, ymax=ymax, color='g', linestyles='dashed', label='rebalance')
    #ax.vlines(x=list(values_concat[values_concat.position_changed == +1].index), ymin=ymin, ymax=ymax, color='b', linestyles='dashed', label='enter')
    #ax.vlines(x=list(values_concat[values_concat.position_changed == -1].index), ymin=ymin, ymax=ymax, color='r', linestyles='solid', label='leave')
    
    plt.show()

NameError: name 'symbols_sets' is not defined